In [ ]:
#this does not work - something about the loss function is not making a difference

In [1]:
!unzip dsb-24-german-credit.zip

Archive:  dsb-24-german-credit.zip
  inflating: german_credit_test.csv  
  inflating: german_credit_test_submission.csv  
  inflating: german_credit_train.csv  


In [1]:
import pandas as pd
df = pd.read_csv('german_credit_train.csv')

In [2]:
numerical_cols = [
    'LoanDuration',
    'LoanAmount',
    'InstallmentPercent',
    'CurrentResidenceDuration',
    'Age',
    'ExistingCreditsCount',
]
categorical_cols = [
  'CheckingStatus',
  'CreditHistory',
 'LoanPurpose',
 'ExistingSavings',
 'EmploymentDuration',
 'Sex',
 'OthersOnLoan',
 'OwnsProperty',
 'InstallmentPlans',
 'Housing',
 'Job',
 'Dependents',
 'Telephone',
 'ForeignWorker',]

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# Load dataset (replace with actual data)

# Define target and features
target = "Risk"
X = df.drop(columns=[target])
y = df[target]

# Binary encode target variable if classification
y = LabelEncoder().fit_transform(y) if y.nunique() == 2 else y

# Identify categorical and numerical columns
cat_cols = categorical_cols
num_cols = numerical_cols

# Preprocessing for numerical data
num_transformer = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
cat_transformer = Pipeline([
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])


In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
def compute_costs(LoanAmount):
     return({'Risk_No Risk': 5.0 + .6 * LoanAmount, 'No Risk_No Risk': 1.0 - .05 * LoanAmount,
         'Risk_Risk': 1.0, 'No Risk_Risk': 1.0})
def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
   '''
   A custom metric for the German credit dataset
   '''
   real_prop = {'Risk': .02, 'No Risk': .98}
   train_prop = {'Risk': 1/3, 'No Risk': 2/3}
   custom_weight = {'Risk': real_prop['Risk']/train_prop['Risk'], 'No Risk': real_prop['No Risk']/train_prop['No Risk']}
   costs = compute_costs(solution['LoanAmount'])
   y_true = solution['Risk']
   y_pred = submission['Risk']
   loss = (y_true=='Risk') * custom_weight['Risk'] *\
               ((y_pred=='Risk') * costs['Risk_Risk'] + (y_pred=='No Risk') * costs['Risk_No Risk']) +\
            (y_true=='No Risk') * custom_weight['No Risk'] *\
               ((y_pred=='Risk') * costs['No Risk_Risk'] + (y_pred=='No Risk') * costs['No Risk_No Risk'])
   return loss.mean()


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# Load dataset (replace with actual data)

# Define target and features
target = "Risk"
X = df.drop(columns=[target])
y = df[target]

# Binary encode target variable if classification
y = LabelEncoder().fit_transform(y) if y.nunique() == 2 else y

# Identify categorical and numerical columns
cat_cols = categorical_cols
num_cols = numerical_cols

# Preprocessing for numerical data
num_transformer = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
cat_transformer = Pipeline([
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

# Choose model type (classification or regression)
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbose=1
                      )

# Create full pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#for final run
X_train, y_train = X, y


#this can be a way to find the right balance.
#class_weights = {0: 0.8, 1: 0.2}
#sample_weights = [class_weights[label] for label in y_train]


# Define parameter grid for hyperparameter tuning
param_grid = {
    'model__n_estimators': [400, 500, 600, 700, 800],
    'model__max_depth': [2, 3, 5, 7],
    'model__learning_rate': [0.005, 0.01, 0.1, 0.2]
}

# Perform grid search - custom scoring loss function here?
search = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
search.fit(X_train, y_train,
           #model__sample_weight=sample_weights
           )

# Evaluate model
y_pred = search.best_estimator_.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"Accuracy: {score:.4f}")

# Best parameters
print("Best parameters:", search.best_params_)


/Users/chwang/anaconda3/envs/mlenv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [12:57:44] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_40lf8y233v/croot/xgboost-split_1728569127361/work/cpp_src/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/chwang/anaconda3/envs/mlenv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [12:57:44] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_40lf8y233v/croot/xgboost-split_1728569127361/work/cpp_src/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/chwang/anaconda3/envs/mlenv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [12:57:44] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_40lf8y233v/croot/xgboost-split_1728569127361/work/cpp_src/src/learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/chwang/anaconda

Accuracy: 0.8413
Best parameters: {'model__learning_rate': 0.005, 'model__max_depth': 7, 'model__n_estimators': 400}


In [52]:
# prompt: open the german credit test.csv, and use the above gridsearch model + preprocessing pipeline to generate predictions the test.csv

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error


# Load the test dataset
try:
    test_df = pd.read_csv('german_credit_test.csv')
except FileNotFoundError:
    print("Error: 'german_credit_test.csv' not found. Please make sure the file exists and the name is correct.")
    exit() # Exit the script if the file is not found
except Exception as e:
    print(f"An error occurred while loading the test data: {e}")
    exit()

# Assuming you have already defined the following from the previous code block:
# numerical_cols, categorical_cols, target, preprocessor, model, pipeline, search (with the fitted model)

# Prepare the test data
X_test_final = test_df.drop(columns=[target], errors='ignore') # Drop target column if it exists

# Generate predictions
y_pred_final = search.best_estimator_.predict(X_test_final)


# Optionally save predictions to a file
predictions_df = pd.DataFrame({
    'Id': test_df.index,
    'Risk': y_pred_final}
                              )
predictions_df.to_csv('german_credit_test_predictions.csv', index=False)


In [53]:
# prompt: Id,TARGET
# 1, "Risk"
# 2, "No Risk"
# 3, "Risk"
# etc.
# is the format of the predictions df that i need.

import pandas as pd
# ... (Your existing code)

# Inverse transform the predictions if necessary
le = LabelEncoder()
le.fit(df['Risk'])
y_pred_final_labels = le.inverse_transform(y_pred_final)


# Optionally save predictions to a file
predictions_df = pd.DataFrame({
    'Id': test_df.index,
    'Risk': y_pred_final_labels # Use 'TARGET' as column name
})
#print(predictions_df['TARGET'].value_counts())
predictions_df.to_csv('german_credit_test_predictions.csv', index=False)


In [7]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# 🔹 Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Define Features (Exclude loan_amount for separate handling)
num_features = numerical_cols
cat_features = categorical_cols

# 🔹 Preprocessing Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

# 🔹 Custom Loss Function
def custom_loss(y_true, y_pred):
    """
    Custom loss function for LightGBM.
    """

    #y_true = y_true.get_label()
    y_pred = y_pred.get_label()
    y_pred = np.clip(y_pred, 1e-6, 1 - 1e-6)  # Prevent log(0)

    # Convert raw predictions to probabilities
    y_pred = 1 / (1 + np.exp(-y_pred))

    # Get loan amounts from the global context
    loan_amounts = custom_loss.loan_amounts

    # Compute per-sample loss matrices
    loss_matrices = np.array([compute_loss_matrix(amt) for amt in loan_amounts])

    # Calculate losses
    fn_loss = loss_matrices[:, 1, 0] * y_true * np.log(y_pred)  # False Negative
    fp_loss = loss_matrices[:, 0, 1] * (1 - y_true) * np.log(1 - y_pred)  # False Positive

    loss = -(fn_loss + 3*fp_loss)  # Total loss

    # Compute gradients and Hessians
    grad = -(loss_matrices[:, 1, 0] * y_true / y_pred - loss_matrices[:, 0, 1] * (1 - y_true) / (1 - y_pred))
    hess = loss_matrices[:, 1, 0] * y_true / (y_pred ** 2) + loss_matrices[:, 0, 1] * (1 - y_true) / ((1 - y_pred) ** 2)

    return grad, hess

# 🔹 Custom Loss Matrix Function
def compute_loss_matrix(loan_amount):
    """
    Returns a loss matrix for a given loan amount.
    - FN cost = 10 × loan amount (very high for false negatives)
    - FP cost = 1 + (loan amount / 10,000) (scaled false positive penalty)
    """
    return np.array([[1.0 - .05 * loan_amount, 5.0 + .6 * loan_amount],  # True Negative, False Positive
                     [1.0, 1.0]])  # False Negative, True Positive  # False Negative, True Positive

class LGBMWithCustomLoss(lgb.LGBMClassifier):
    def fit(self, X, y, **kwargs):
        """ Custom fit function that applies a loss matrix per sample. """
        loan_amounts = X['LoanAmount'].values  # Extract per-sample feature
        X_transformed = preprocessor.fit_transform(X)  # Process other features

        # Store loan amounts in the custom loss function
        custom_loss.loan_amounts = loan_amounts

        # Create LGBM dataset
        dtrain = lgb.Dataset(X_transformed, label=y)

        param_dict = self.get_params()

        # Train with custom loss function
        self._model = lgb.train(
            params=param_dict,
            train_set=dtrain,
            num_boost_round=self.n_estimators,
            #objective=custom_loss,  # Custom objective function
            #valid_sets=[dtrain],
            #early_stopping_rounds=10  # Optional
        )
        self.fitted_ = True
        self._n_features = X_transformed.shape[1]
        return self
    def predict_proba(self, X):
        #check_is_fitted(self, 'fitted_')
        X_transformed = preprocessor.transform(X)
        preds = self._model.predict(X_transformed)
        return np.column_stack([1 - preds, preds])

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

# 🔹 Full Pipeline (Loan Amount Handled Separately)
pipeline = Pipeline([
    ('classifier', LGBMWithCustomLoss(objective=custom_loss, n_estimators=100))
])

# 🔹 Train Model
pipeline.fit(X_train, y_train)

# 🔹 Predict & Evaluate
#X_test_transformed = preprocessor.transform(X_test)  # Process test data
#y_pred_proba = pipeline.named_steps['classifier'].predict_proba(X_test_transformed)[:, 1]  # Get probabilities

y_pred = pipeline.named_steps['classifier'].predict(X_test)

# 🔹 Compute Custom Loss on Test Data
score = accuracy_score(y_test, y_pred)
print(f"Accuracy: {score:.4f}")



/Users/chwang/anaconda3/envs/mlenv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000553 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 484
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 61
[LightGBM] [Info] Using self-defined objective function
Accuracy: 0.6400


In [55]:
# prompt: open the german credit test.csv, and use the above gridsearch model + preprocessing pipeline to generate predictions the test.csv

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error


# Load the test dataset
try:
    test_df = pd.read_csv('german_credit_test.csv')
except FileNotFoundError:
    print("Error: 'german_credit_test.csv' not found. Please make sure the file exists and the name is correct.")
    exit() # Exit the script if the file is not found
except Exception as e:
    print(f"An error occurred while loading the test data: {e}")
    exit()

# Assuming you have already defined the following from the previous code block:
# numerical_cols, categorical_cols, target, preprocessor, model, pipeline, search (with the fitted model)

# Prepare the test data
X_test_final = test_df.drop(columns=[target], errors='ignore') # Drop target column if it exists

# Generate predictions
y_pred_final = pipeline.predict(X_test_final)


# Optionally save predictions to a file
predictions_df = pd.DataFrame({
    'Id': test_df.index,
    'Risk': y_pred_final}
                              )
predictions_df.to_csv('german_credit_test_predictions.csv', index=False)


In [56]:
predictions_df.Risk.value_counts()

,count
Risk,
0,1001
